<br>

# SidraPy

- https://github.com/AlanTaranti/sidrapy
- https://sidrapy.readthedocs.io/en/latest

In [ ]:
#!pip install sidrapy --upgrade

In [ ]:
import sidrapy
import pandas as pd

<br>

## Exemplo

In [ ]:
#sidrapy.get_table(table_code="1419", territorial_level="1", ibge_territorial_code="all", period="last 12")

<br>

## Estimativa População

In [ ]:
def get_estimated_population(cod_ibge):
    # Get Table
    df = sidrapy.get_table(
        table_code='6579',
        territorial_level='6',
        ibge_territorial_code=cod_ibge,
        period='all',
        header='n',
    )

    # Dict
    dict_col = {
        'D1C': 'id_municipio',
        'D1N': 'municipio_nome',
        'V': 'n_habitantes',
        'D2N': 'ano'
    }

    # Remane Columns
    df.rename(
        dict_col,
        axis=1,
        inplace=True
    )

    # Select Columns
    df = df[[v for k,v in dict_col.items()]]
    
    # Adjust Columns
    df.sort_values(by=['ano'], inplace=True)
    df['id_municipio'] = pd.to_numeric(df['id_municipio'], errors='coerce')
    df['n_habitantes'] = pd.to_numeric(df['n_habitantes'], errors='coerce')
    df['ano'] = pd.to_numeric(df['ano'], errors='coerce')

    # Results
    return df

In [ ]:
cod_ibge = '3526902' # Limeira

df = get_estimated_population(cod_ibge)
print(df.dtypes)
df.head()

<br>

# Export

In [ ]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [ ]:
input_filepath = os.path.join(os.getcwd(), 'sidrapy.ipynb')
output_filepath = os.path.join(os.getcwd(), 'estimated_population.py')
print(output_filepath)

In [ ]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True 
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(input_filepath)

# Write to output html file
with open(output_filepath,  'w', encoding='utf-8') as f:
    f.write(body)